In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# install konlpy & mecab

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

# 라이브러리 불러오기

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter #리스트 내 중복 개수 세기
from tqdm import tqdm

import re
from konlpy.tag import Mecab

#데이터 불러오기

In [6]:
PATH = '' # PATH 수정하기
train = pd.read_csv(PATH + '실습용 자료/1. 실습용자료.txt', sep = '|', engine = 'python', index_col = 'AI_id',  encoding='cp949')
test = pd.read_csv(PATH + '2. 모델개발용자료.txt', sep = '|', engine = 'python', index_col = 'AI_id',  encoding='cp949')

In [7]:
train.fillna('', inplace = True)
test.fillna('', inplace = True)

In [8]:
train['text'] = train['text_obj'] + ' ' + train['text_mthd'] + ' ' + train['text_deal']
test['text'] = test['text_obj'] + ' ' + test['text_mthd'] + ' ' + test['text_deal']

In [9]:
pattern = '[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》ㄱ-ㅎ|ㅏ-ㅣ]+'
# 한글자 오타 및 특수문자 제거
train['text'] = train['text'].str.replace(pat = pattern, repl = ' ', regex = True)
test['text'] = test['text'].str.replace(pat = pattern, repl = ' ', regex = True)
#영어 대문자로
train['text'] = train['text'].str.upper()
test['text'] = test['text'].str.upper()

# 일반 명사만 저장

In [10]:
def noun_nng(df, col): 
    #mecab 불러오기
    mecab = Mecab()
    
    nouns = []
    for text in tqdm(df[col]):
        noun = mecab.pos(text) #pos 확인
        nng = []
        for i in range(len(noun)):
            if ((noun[i][1]=='NNG') or (noun[i][1]=='NNP') or (noun[i][1]=='NP')): #NNP, NP, NNG인 경우 nouns에 append
                nng.append(noun[i][0])
        if len(nng) == 0:
            nouns.append([text])
        else:
            nouns.append(nng)
    
    nounsToStr = [] 
    for i in range(len(nouns)):
        nounsToStr.append(','.join(nouns[i]).replace(',', ' ')) # string 형태로 저장
    
    nounsToStr = pd.DataFrame(nounsToStr, columns = ['text'])
    
    return nounsToStr


In [11]:
X_train = noun_nng(train, 'text')

100%|██████████| 1000000/1000000 [01:25<00:00, 11748.42it/s]


In [12]:
X_test = noun_nng(test, 'text')

100%|██████████| 100000/100000 [00:07<00:00, 13174.98it/s]


In [13]:
X_train.index = train.index
train['text'] = X_train['text']
X_test.index = test.index
test['text'] = X_test['text']

In [14]:
train.to_csv('train_pre.csv')
test.to_csv('test_pre.csv')